In [128]:
import pandas as pd
import numpy as np
import pickle
from Bio import SeqIO
from Bio import Phylo
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import zlib

# Remove recombinations from MSA

+ input: MSA (fa, xmfa), tree, predicted importations 
+ output 1: MSA without recombination
    + fasta
    + phylip
+ output 2: table of protein ids assigned to importations 


# Results for the test sets

## MSA lengths
+ rso-test (fjat) = 3,825,198 (xmfa: 7,666,198)
+ phy2 = 2,987,190 (xmfa: 6,086,190)
+ phy4 = 3,219,942
+ phy3 = 3,459,843 (xmfa: 6,947,843)

(for ASM359060v in phy2 dataset the summed length is 116,425)

## Summed recombinations length
+ rso = 92,925
+ phy2 = 290,218
+ phy4 = 58,063
+ phy3 = 120,259

(phy4 strain PSI07 is from Indonesia! very different)

In [129]:
SAMPLE = 'korea'#'rso_test'

recombis = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_cfml/rso_test_cfml.importation_status.txt'
path2tree = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_cfml/rso_test_cfml.labelled_tree.newick'
xm_msa_in = "/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_concat.xmfa"
fa_msa_in = "/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_concat.fasta"
accDB = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/MSA_ACC_DB_rso_test.pkl'
protDB = '../'+SAMPLE+'_outputs/GENOME_PROTEIN_DB_'+SAMPLE+'.pkl'
# out
fa_msa_out = "/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_concat_no_RR.fasta"
ph_msa_out = "/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_concat_no_RR.phylip"
updated_recombis = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_recombis2prot.csv'
mult_recombis = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_recombis_mult_genes.txt'
top_recombis = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_recombis_top_genes.txt'


# in
recombis = recombis.replace("rso_test", SAMPLE)
path2tree = path2tree.replace("rso_test", SAMPLE)
xm_msa_in = xm_msa_in.replace("rso_test", SAMPLE)
fa_msa_in = fa_msa_in.replace("rso_test", SAMPLE)
accDB = accDB.replace("rso_test", SAMPLE)
protDB = protDB.replace("rso_test", SAMPLE)
# out
fa_msa_in = fa_msa_in.replace("rso_test", SAMPLE)
ph_msa_out = ph_msa_out.replace("rso_test", SAMPLE)
updated_recombis = updated_recombis.replace("rso_test", SAMPLE)
mult_recombis = mult_recombis.replace("rso_test", SAMPLE)
top_recombis = top_recombis.replace("rso_test", SAMPLE)

In [ ]:
# in
recombis = snakemake.params["recombis"]
path2tree = snakemake.params["tree"]
xm_msa_in = snakemake.input["xm_msa_in"]
fa_msa_in = snakemake.input["fa_msa_in"]
accDB = snakemake.input["accDB"]
protDB = snakemake.input["protDB"]
# out
fa_msa_out = snakemake.output["fa_msa_out"]
ph_msa_out = snakemake.output["ph_msa_out"]
updated_recombis = snakemake.output["updated_recombis"]
mult_recombis = snakemake.output["mult_recombis"]
top_recombis = snakemake.output["top_recombis"]

# Replace the inner nodes with the leaves 

In [130]:
imp_stat = pd.read_csv(recombis, sep='\t')
imp_stat

,Node,Beg,End
0,pe13,1951157,1951163
1,pe13,3007423,3007452
2,pe13,3008403,3008431
3,pe13,3008682,3008688
4,pe13,5281084,5281091
...,...,...,...
677,NODE_13,6402045,6406018
678,NODE_13,6406361,6410314
679,NODE_13,6410572,6412423
680,NODE_13,6413081,6413086


In [131]:
for tree in Phylo.parse(path2tree, 'newick'):
    parents = {}
    # iterate through inner nodes 
    for clade in tree.get_nonterminals(order="level"):
        parents[clade.name] = []
        # get leaves for each inner node
        for child in clade.get_terminals():
            parents[clade.name].append(child.name)
        #parents[clade.name] = ";".join(parents[clade.name])
parents

{'NODE_15': ['pe26', 'pe61', 'to53', 'pe15', 'pe30', 'pe9', 'to28', 'pe13'],
 'NODE_14': ['pe26', 'pe61', 'to53', 'pe15', 'pe30', 'pe9', 'to28'],
 'NODE_13': ['pe26', 'pe61', 'to53', 'pe15', 'pe30', 'pe9'],
 'NODE_12': ['pe26', 'pe61', 'to53', 'pe15', 'pe30'],
 'NODE_10': ['pe26', 'pe61', 'to53'],
 'NODE_11': ['pe15', 'pe30'],
 'NODE_9': ['pe26', 'pe61']}

In [132]:
new_len = len(imp_stat)

for p in parents:

    is_node = imp_stat["Node"].str.startswith(p)
    num_child = len(parents[p]) 
    print(p,'has',len(imp_stat[is_node]),'recombinations and',num_child,'children')
    new_len = new_len + len(imp_stat[is_node]) * (num_child - 1)
    
    #imp_stat = imp_stat.append([imp_stat[is_node]]*(num_child-1), ignore_index=True)
    print()
    
    # for each child: duplicate the parent row
    for child in parents[p]:       
        node_imp = imp_stat[is_node].replace({p:child}) 
        imp_stat = imp_stat.append([node_imp], ignore_index=True)
        is_node = imp_stat["Node"].str.startswith(p)

# remove the remaining parent rows

imp_stat = imp_stat.loc[~imp_stat["Node"].str.contains("NODE_")]
print('DF length after parent nodes expantion =', len(imp_stat), '; Must be =', new_len)
assert(new_len == len(imp_stat))

NODE_15 has 0 recombinations and 8 children

NODE_14 has 0 recombinations and 7 children

NODE_13 has 5 recombinations and 6 children

NODE_12 has 0 recombinations and 5 children

NODE_10 has 50 recombinations and 3 children

NODE_11 has 6 recombinations and 2 children

NODE_9 has 199 recombinations and 2 children

DF length after parent nodes expantion = 1012 ; Must be = 1012


# Sort and sum the importations

In [133]:
imp_stat = imp_stat.sort_values(by=['Beg', 'End'])
#imp_stat

In [134]:
beg = imp_stat["Beg"].values.copy()
end = imp_stat["End"].values.copy()

In [135]:
ranges2del = []
for i in range(len(imp_stat)):
    ranges2del.extend(range(beg[i]-1, end[i]))
ranges2del = list(dict.fromkeys(ranges2del))
print('Summed recombination length:', len(ranges2del))

Summed recombination length: 53735


# Fasta vs. XMFA positions mapping list

* list of updated positions 
* index represent Fasta sequence positions 
* values represent XMFA sequence positions 

In [136]:
# parse the genes MSAs into vector of the alignments' lengths
aln_length = []
aln = open(xm_msa_in, 'r')

for l in aln:
    
    if l.startswith('>'):
        c = 0
    elif l.startswith('='):
        aln_length.append(c-1) # minus one char for the \n symbol
    else:
        c = c + len(l)
        
print('Found genes MSAs / LCBs:', len(aln_length))    
aln.close()

Found genes MSAs / LCBs: 3996


In [137]:
# insert 1000 count after each gene MSA
index_mapping = list(range(0,aln_length[0]))
sum_lengths = aln_length[0]

for i in range(1,len(aln_length)):  
    index_mapping.extend(range(sum_lengths + i*1000, sum_lengths + aln_length[i] + i*1000))
    sum_lengths = sum_lengths + aln_length[i]
    

In [138]:
print('Fasta length', sum(aln_length), '\nXMFA length', sum(aln_length)+(len(aln_length)-1)*1000)
# index = fasta file length
assert(sum(aln_length) == len(index_mapping))
# values = xmfa file length
assert(sum(aln_length)+(len(aln_length)-1)*1000 == index_mapping[-1]+1) 

Fasta length 4089060 
XMFA length 8084060


# Get the fasta position of each insertion from the xmfa position

+ use ranges to improve the running time of .index(i, start, end)

In [139]:
fa_beg = []
fa_end = []
previous_beg = 0
previous_end = 0
L=len(index_mapping)

for r_i in range(len(imp_stat)):
    
    # -1 for 0-start indexing
    r_beg = beg[r_i] - 1
    r_end = end[r_i] - 1
    #print(r_beg, r_end)
    
    # USE PREVIOUS_BEG FOR BOTH BECAUSE OF THE SORTING ISSUE
    r_beg_index = index_mapping.index(r_beg, previous_beg, L)#r_beg+1)
    r_end_index = index_mapping.index(r_end, previous_beg, L)#r_end+1)
    
    previous_beg = r_beg_index
    previous_end = r_end_index
    
    fa_beg.append(r_beg_index)
    fa_end.append(r_end_index)
    
    #print(r_i,"xmfa - fa:",r_beg_index, r_end_index)

imp_stat['FA Beg'] = fa_beg
imp_stat['FA End'] = fa_end

In [140]:
imp_stat['debug_len_must'] = imp_stat['End'] - imp_stat['Beg']
imp_stat["debug_len_is"] = imp_stat['FA End'] - imp_stat['FA Beg']
#assert(len(imp_stat.loc[imp_stat['debug_len_must'] != imp_stat['debug_len_is']]) == 0)
#imp_stat
imp_stat.loc[imp_stat['debug_len_must'] != imp_stat['debug_len_is']].to_csv(mult_recombis, index=False)

# Delete the recombinations from the fasta file 

In [141]:
fasta_out = open(fa_msa_out, 'w')
num_of_strains = 0
for record in SeqIO.parse(fa_msa_in, 'fasta'):
    num_of_strains = num_of_strains + 1
    fasta_out.write('>' + record.id + '\n')
    genome_imp_stat = imp_stat.loc[imp_stat['Node'] == record.id]
    beg = genome_imp_stat["FA Beg"].values.copy()
    end = genome_imp_stat["FA End"].values.copy()
    
    ranges2del = []
    for i in range(len(genome_imp_stat)):
        ranges2del.extend(range(beg[i]-1, end[i]))
    ranges2del = list(dict.fromkeys(ranges2del))
    
    print('Genome',record.id,'has',len(ranges2del),'recombinations')
    #print(ranges2del)
    
    print('Number of gaps before',record.seq.count('-'))
    seq = list(record.seq)
    for to_del in ranges2del:
        seq[to_del] = '-'
    seq = ''.join(seq)
    print('Number of gaps after',seq.count('-'))
    print()
    fasta_out.write(seq + '\n')
    
fasta_out.close()

Genome pe13 has 360 recombinations
Number of gaps before 33132
Number of gaps after 33401

Genome pe15 has 10651 recombinations
Number of gaps before 33003
Number of gaps after 43207

Genome pe26 has 38512 recombinations
Number of gaps before 44475
Number of gaps after 71541

Genome pe30 has 10543 recombinations
Number of gaps before 33639
Number of gaps after 43753

Genome pe61 has 38959 recombinations
Number of gaps before 39042
Number of gaps after 76904

Genome pe9 has 9857 recombinations
Number of gaps before 34092
Number of gaps after 43697

Genome to28 has 401 recombinations
Number of gaps before 29634
Number of gaps after 29967

Genome to53 has 20680 recombinations
Number of gaps before 46062
Number of gaps after 65889



# Convert to Phylip TODO:length!!!

In [143]:
#SeqIO.convert(fa_msa_out, "fasta", ph_msa_out, "phylip")

nexus = open(ph_msa_out, 'w')
nexus.write('#NEXUS\n')
nexus.write('begin data;\n')
nexus.write('dimensions ntax='+str(num_of_strains)+' nchar='+str(sum(aln_length))+' ;\n')
nexus.write('format datatype=dna ;\n')
nexus.write('matrix\n')

for record in SeqIO.parse(fa_msa_out, "fasta"):
    nexus.write(record.description+'\t'+str(record.seq)+'\n')
    
nexus.write(';\n')
nexus.write('end;\n')

nexus.close()

# Expand and export the importation_status table

In [144]:
# read Accession Database

with open(accDB, 'rb') as handle:
    acc = pickle.load(handle)
acc.keys()

dict_keys(['pe13', 'pe15', 'pe26', 'pe30', 'pe61', 'pe9', 'to28', 'to53'])

In [145]:
# read Protein Database

prot_db = pd.read_pickle(protDB)
prot_db = prot_db.rename(columns=lambda x: str(x).split('$')[0])
prot_db

,to53,pe15,pe13,pe61,to28,pe9,pe26,pe30
GDEOEPAP_00001,"{'chr': '1', 'location': (30, 31, 32, 33, 34, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
GDEOEPAP_00002,"{'chr': '1', 'location': (1876, 1877, 1878, 18...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
GDEOEPAP_00003,"{'chr': '1', 'location': (3128, 3129, 3130, 31...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
GDEOEPAP_00004,"{'chr': '1', 'location': (7139, 7138, 7137, 71...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
GDEOEPAP_00005,"{'chr': '1', 'location': (8581, 8582, 8583, 85...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
EENJCLCO_05336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'chr': 'cluster_012', 'location': (8609, 8610..."
EENJCLCO_05337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'chr': 'cluster_012', 'location': (10200, 101..."
EENJCLCO_05338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'chr': 'cluster_012', 'location': (11858, 118..."
EENJCLCO_05339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'chr': 'cluster_012', 'location': (12330, 123..."


for k in acc.keys():
    g2ranges = imp_stat.loc[imp_stat['Node'] == k]
    
    g_beg = g2ranges["FA Beg"].values.copy()
    g_end = g2ranges["FA End"].values.copy()
    
    print(k)
    for i in range(len(g_beg)):
        print(set(acc[k][g_beg[i]:g_end[i]]))

In [146]:
acc.keys()

dict_keys(['pe13', 'pe15', 'pe26', 'pe30', 'pe61', 'pe9', 'to28', 'to53'])

In [147]:
# assign improtations to proteins

g_beg = imp_stat["FA Beg"].values.copy()
g_end = imp_stat["FA End"].values.copy()
g_node = imp_stat["Node"].values.copy()

import_acc = []
import_products = []
for i in range(len(g_beg)):
    import_acc.append(set(acc[g_node[i]][g_beg[i]:g_end[i]]))
    
    products = []
    for a in import_acc[-1]:
        products.append(zlib.decompress(prot_db.loc[a, g_node[i]]['product']).decode())
    import_products.append(products)
    
imp_stat['Prot-ACC'] = import_acc
imp_stat['Product'] = import_products

In [148]:
imp_stat.to_csv(updated_recombis, index=False)
imp_stat

,Node,Beg,End,FA Beg,FA End,debug_len_must,debug_len_is,Prot-ACC,Product
276,to53,6423,6437,4422,4436,14,14,{GDEOEPAP_00003},[[DNA gyrase subunit B]]
712,pe26,6671,6680,4670,4679,9,9,{NCAEKIBJ_00003},[[DNA gyrase subunit B]]
762,pe61,6671,6680,4670,4679,9,9,{KPJNPNMP_00003},[[DNA gyrase subunit B]]
812,to53,6671,6680,4670,4679,9,9,{GDEOEPAP_00003},[[DNA gyrase subunit B]]
874,pe26,6839,6941,4838,4940,102,102,{NCAEKIBJ_00003},[[DNA gyrase subunit B]]
...,...,...,...,...,...,...,...,...,...
9,pe13,8024479,8024551,4063478,4063550,72,72,{AIELFPIA_05293},[[hypothetical protein]]
1071,pe26,8025703,8025774,4063702,4063773,71,71,{NCAEKIBJ_05096},[[hypothetical protein]]
1270,pe61,8025703,8025774,4063702,4063773,71,71,{KPJNPNMP_05153},[[hypothetical protein]]
1072,pe26,8072284,8072490,4082283,4082489,206,206,{NCAEKIBJ_05125},[[hypothetical protein]]


In [149]:
imp_stat = pd.read_csv(updated_recombis)
imp_stat = imp_stat.drop(['Node'], axis=1).drop_duplicates()
imp_stat['Product'].value_counts(normalize=True)[:50].to_csv(top_recombis)

# TODO {'WP_071623447.1'} seq length is less than improtation length?
# TODO enrichment with probabilities 
e.g. numOf(T3 effectors with recombinations)/numOf(all T3 effectors in the genome)

In [150]:
#prot_db.loc["WP_071507848.1","ASM1330693"]